# Tools for a Customer Outreach(拓展) Campaign(活动)

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

# ========== 艾可配置（替换 OpenAI） ==========
# 1. aic API Key（替换成你的实际密钥）
os.environ["OPENAI_API_KEY"] = "sk-ESFKBhq269g1lgBLmAhJCvA3zQ4WxHK4yxkapD2HI5rf3Pb1"
os.environ["OPENAI_BASE_URL"] = "https://api.aicanapi.com"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4-turbo"